<h3> Devoir 2 - a rendre le 5 Janvier 2018 </h3>

Le devoir comprend 3 questions a traiter en vous inspirant (librement) du notebook ci-dessous.
Vous pouvez egalement lire le tres bon Chapitre 2 de notre livre "hands-on ..." pour plus de 
perspectives et plus de details. Le notebook ci-dessous est essentiellement une simplification 
de ce chapitre adaptee a notre jeu de donnees.

<ol>
<li> 
Construire un regresseur lineaire pour predire le resultat au devoir 1 et calculer son score. On choisira environ 5 personnes dans le groupe test tire au hasard, le reste de la classe servant a fournir des donnees au regresseur.
</li>
<li> 
Ajouter aux attributs existants dans les donnees une combinaison d'attributs de votre choix. Vous trouverez des exemples ci-dessous. Comparer les scores du predicteur avec et sans ces (combinaisons d') attributs supplementaires.
</li>
<li> 
Choisir des partitions train/test qui engendrent des predicteurs differents en selectionnant un groupe test de maniere appropriee. Par exemple, on pourra imaginer que l'algorithme serve a creer les notes des etudiants absents a l'examen et essayer de maximiser ces predictions de notes. (NB: bien sur ceci est strictement a ne pas faire dans un veritable cas d'analyse de donnees!)
</li>
</ol>

<h5>Bonifications:</h5>

Pour ceux qui le desirent, des points supplementaires sont prevus pour le traitement de l'une des questions supplementaires ci-dessous.
<ol>
<li>
Automatiser le traitement des donnees par des fonctions
</li>
<li> 
Utiliser un autre modele statistique que le regresseur lineaire
</li>
<li> 
Etendre les donnees: eg 1) avec les notes du cours de chimie, 2) nb d'amis facebook, 3) ...
</li>
<li> 
Utiliser les hyperparametres de la <a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression">classe LinearRegression de scikitlearn</a> pour ameliorer le modele.
</li>
</ol> 

<h3> NB </h3>

N'oubliez pas de bien commenter votre source pour en faciliter la lecture.

Utilisez le fichier de donnees disponibles sur le github (j'y ai corrige quelques aberrations du csv extrait du document google sheet).


<h3> chargement des donnees </h3>

In [2]:
# Common imports
import numpy as np
import os
import pandas as pd

In [3]:
# to make this notebook's output stable across runs
np.random.seed(42)

In [5]:
AZOTE_PATH = os.path.join("..","datasets")
AZOTE_FILE = "l.csv"
dc = pd.read_csv(os.path.join(AZOTE_PATH,AZOTE_FILE))

In [6]:
# pour obtenir la liste "cut-and-paste ready" des noms des colonnes
list(dc)

['name.familyName',
 'name.givenName',
 'primaryEmail',
 'extraversion\n0 to 9',
 'session 1\ntest /10',
 'session 2\ntest /10',
 'session 3\ntest /10',
 'Ressenti \nséances\n2 et 3 test /4',
 'Devoir 1\ntest /10']

we change the names of the columns for convenience

In [7]:
#cd.drop('primaryEmail', axis=1, inplace=True)
dc = dc.rename(columns={"name.familyName":"nom",'name.givenName':"prenom",
 'primaryEmail':"email",
 'extraversion\n0 to 9':"ex",
 'session 1\ntest /10':"s1",
 'session 2\ntest /10':"s2",
 'session 3\ntest /10':"s3",
 'Ressenti \nséances\n2 et 3 test /4':"r23",
 'Devoir 1\ntest /10':"d1"})

<h3> data visualisation </h3>

In [8]:
list(dc)

['nom', 'prenom', 'email', 'ex', 's1', 's2', 's3', 'r23', 'd1']

la colonne "ressenti" a beaucoup de valeurs manquantes, but apart from that we 
are good as we can see from the info method

In [9]:
dc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 9 columns):
nom       28 non-null object
prenom    28 non-null object
email     28 non-null object
ex        28 non-null int64
s1        28 non-null int64
s2        28 non-null int64
s3        28 non-null int64
r23       7 non-null float64
d1        28 non-null int64
dtypes: float64(1), int64(5), object(3)
memory usage: 2.0+ KB


In [10]:
dc["r23"].value_counts()

3.0    4
2.0    3
Name: r23, dtype: int64

In [11]:
dc.describe()

,ex,s1,s2,s3,r23,d1
count,28.000000,28.000000,28.000000,28.000000,7.000000,28.00000
mean,6.464286,5.178571,5.107143,6.857143,2.571429,8.50000
std,1.643892,2.037674,1.872595,1.432798,0.534522,0.96225
min,3.000000,1.000000,0.000000,4.000000,2.000000,6.00000
25%,5.750000,3.750000,4.000000,6.000000,2.000000,8.00000
50%,6.500000,5.500000,5.000000,7.000000,3.000000,8.50000
75%,8.000000,7.000000,6.000000,8.000000,3.000000,9.00000
max,10.000000,8.000000,8.000000,10.000000,3.000000,10.00000


In [12]:
correlation_total = dc.corr()
correlation_total["d1"].sort_values(ascending=False)

d1     1.000000
s3     0.349226
s2     0.298038
s1     0.292782
ex     0.011707
r23   -0.416667
Name: d1, dtype: float64

In [13]:
dc.head()

,nom,prenom,email,ex,s1,s2,s3,r23,d1
0,ARNAUD BOISSEL,Candice,candice.arnaudboissel@students.cri-paris.org,6,3,0,5,2.0,8
1,BACCAINI,Lucien,lucien.baccaini@students.cri-paris.org,8,7,6,8,NaN,8
2,BALY,Antoinette,antoine.baly@students.cri-paris.org,8,5,5,6,2.0,9
3,BELLENGIER,Juliette,juliette.bellengier@students.cri-paris.org,7,6,6,7,NaN,8
4,BEN ALI,Raja,raja.benali@students.cri-paris.org,6,8,6,10,NaN,10


<h3> separation des donnees </h3>

On separe nos donnees en deux: 1) un ensemble de donnees d'apprentissage a utiliser tout de suite et 2) un ensemble de donnees auxquelles on ne touche plus et qui vont nous servir a tester notre "predicteur" lorsque celui-ci sera construit

In [14]:
# Sklearn has train_test_split()
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [15]:
tr_c, tt_c = split_train_test(dc,0.2) # 20% des donnees sont rangees a part
print(len(tr_c), "train +", len(tt_c), "test")

23 train + 5 test


In [16]:
tr_c.head(2)

,nom,prenom,email,ex,s1,s2,s3,r23,d1
12,HENNEBICQ,Léa,lea.hennebicq@students.cri-paris.org,8,4,3,4,NaN,8
17,PICARDAT,Zoé,zoe.picardat@students.cri-paris.org,6,6,6,9,NaN,9


In [17]:
tt_c.head(3)

,nom,prenom,email,ex,s1,s2,s3,r23,d1
9,GASTINEL,Salomé,salome.gastinel@students.cri-paris.org,4,3,3,7,NaN,8
25,Vercambre,Julia,julia.vercambre@students.cri-paris.org,7,8,7,7,NaN,10
8,DAGNOGO,Maelle,maelle.dagnogo@students.cri-paris.org,7,4,3,7,NaN,7


<h3> recherche de signaux </h3>

correlations - on cherche un "signal" qui contient de l'information sur d1

In [18]:
corr_matrix = tr_c.corr()
#print(corr_matrix)

In [19]:
corr_matrix["d1"].sort_values(ascending=False)

d1     1.000000
s3     0.394334
s2     0.169119
s1     0.115399
ex    -0.096976
r23   -0.707107
Name: d1, dtype: float64

ah-ah les bavards sont legerement negativement correles a la note au devoir 1 :)
que devons-nous penser de la correlation de r23??

tentative de recombinaison des attributs pour trouver des signaux additionnels

In [20]:
# abreviations commodes
tr_ex, tt_ex = tr_c["ex"], tt_c["ex"]
tr_s1, tt_s1 = tr_c["s1"], tt_c["s1"]
tr_s2, tt_s2 = tr_c["s2"], tt_c["s2"]
tr_s3, tt_s3 = tr_c["s3"], tt_c["s3"]

on ajoute les nouveaux attributs comme de nouvelles colonnes

In [21]:
# caveat: assign retourne une nouvelle matrice 
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.assign.html
tr_c = tr_c.assign(s21=tr_s2-tr_s1,s32=tr_s3-tr_s2,s31=tr_s3-tr_s1,exx= 5 * tr_ex)
tt_c = tt_c.assign(s21=tt_s2-tt_s1,s32=tt_s3-tt_s2,s31=tt_s3-tt_s1,exx= 5 * tt_ex)

In [22]:
tr_c.head(2)

,nom,prenom,email,ex,s1,s2,s3,r23,d1,exx,s21,s31,s32
12,HENNEBICQ,Léa,lea.hennebicq@students.cri-paris.org,8,4,3,4,NaN,8,40,-1,0,1
17,PICARDAT,Zoé,zoe.picardat@students.cri-paris.org,6,6,6,9,NaN,9,30,0,3,3


In [23]:
tt_c.head(5)

,nom,prenom,email,ex,s1,s2,s3,r23,d1,exx,s21,s31,s32
9,GASTINEL,Salomé,salome.gastinel@students.cri-paris.org,4,3,3,7,NaN,8,20,0,4,4
25,Vercambre,Julia,julia.vercambre@students.cri-paris.org,7,8,7,7,NaN,10,35,-1,-1,0
8,DAGNOGO,Maelle,maelle.dagnogo@students.cri-paris.org,7,4,3,7,NaN,7,35,-1,3,4
21,SUDRE-ROUFFAUX,Louisa,louisa.sudre-rouffaux@students.cri-paris.org,8,7,5,7,NaN,10,40,-2,0,2
0,ARNAUD BOISSEL,Candice,candice.arnaudboissel@students.cri-paris.org,6,3,0,5,2.0,8,30,-3,2,5


In [24]:
# on calcule les nouvelles correlations avec d1
corr_matrix_2 = tr_c.corr()
corr_matrix_2["d1"].sort_values(ascending=False)

d1     1.000000
s3     0.394334
s32    0.186333
s2     0.169119
s31    0.151349
s1     0.115399
s21    0.020058
exx   -0.096976
ex    -0.096976
r23   -0.707107
Name: d1, dtype: float64

Que remarque-t-on? 1) exx a exactement la meme information que ex; c'est l'effet de la normalisation des correlations; 2) voit-on une nouvelle observable interessante?

<h3> Prepare data for Learning </h3>

In [25]:
tr_c_labels, tt_c_labels = tr_c['d1'], tt_c['d1']

In [26]:
tr_c_no_label = tr_c.drop('d1',axis=1) # drop labels for training set - drop creates a *copy*
tt_c_no_label = tt_c.drop('d1',axis=1)

In [27]:
print("",list(tr_c_no_label),"\n",list(tt_c_no_label))

 ['nom', 'prenom', 'email', 'ex', 's1', 's2', 's3', 'r23', 'exx', 's21', 's31', 's32'] 
 ['nom', 'prenom', 'email', 'ex', 's1', 's2', 's3', 'r23', 'exx', 's21', 's31', 's32']


traitement des donnees incompletes - we make up values for the missing ones (in r23)!

In [28]:
# first we need to drop non numerical columns
tr_c_no_label_num = tr_c_no_label.drop(['nom', 'prenom', 'email'],axis=1)
tt_c_no_label_num = tt_c_no_label.drop(['nom', 'prenom', 'email'],axis=1)
print("",list(tr_c_no_label_num),"\n",list(tt_c_no_label_num))

 ['ex', 's1', 's2', 's3', 'r23', 'exx', 's21', 's31', 's32'] 
 ['ex', 's1', 's2', 's3', 'r23', 'exx', 's21', 's31', 's32']


In [29]:
tr_c_no_label_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 12 to 6
Data columns (total 9 columns):
ex     23 non-null int64
s1     23 non-null int64
s2     23 non-null int64
s3     23 non-null int64
r23    6 non-null float64
exx    23 non-null int64
s21    23 non-null int64
s31    23 non-null int64
s32    23 non-null int64
dtypes: float64(1), int64(8)
memory usage: 1.8 KB


In [30]:
tt_c_no_label_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 9 to 0
Data columns (total 9 columns):
ex     5 non-null int64
s1     5 non-null int64
s2     5 non-null int64
s3     5 non-null int64
r23    1 non-null float64
exx    5 non-null int64
s21    5 non-null int64
s31    5 non-null int64
s32    5 non-null int64
dtypes: float64(1), int64(8)
memory usage: 400.0 bytes


Check that this is the same as manually computing the median of each attribute:

In [31]:
tr_c_no_label_num.median()

ex      6.0
s1      6.0
s2      6.0
s3      7.0
r23     3.0
exx    30.0
s21     0.0
s31     1.0
s32     1.0
dtype: float64

In [32]:
tr_c_no_label_num['r23']

12    NaN
17    NaN
22    3.0
11    NaN
13    2.0
15    NaN
1     NaN
4     NaN
5     NaN
2     2.0
16    NaN
23    NaN
3     NaN
26    NaN
24    NaN
18    NaN
27    NaN
20    3.0
7     NaN
10    NaN
14    NaN
19    3.0
6     3.0
Name: r23, dtype: float64

In [33]:
tt_c_no_label_num['r23']

9     NaN
25    NaN
8     NaN
21    NaN
0     2.0
Name: r23, dtype: float64

Because of all the NaNs we need to transform the training set before learning - to this effect, we use a primitive from scikitlearn to deal with bad/missing values (NaN's); read this to see why it is important 
http://scikit-learn.org/stable/modules/preprocessing.html#imputation

In [34]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values="NaN",strategy="median")

"fit" is learning what to replace NaNs' with - here the medians
see hands-on "chapter 2/Data Cleaning"

In [35]:
imp.fit(tr_c_no_label_num)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

accessible like so:

In [66]:
list(imp.statistics_)

[6.0, 6.0, 6.0, 7.0, 3.0, 30.0, 0.0, 1.0, 1.0]

In [1]:
def ppa(array,num=10):
    b = min(num,len(array))
    for i in range(0,b):
        print(array[i])

In [37]:
X = imp.transform(tr_c_no_label_num) # a numpy array
ppa(X,5) # we see that the "filling in" is by median indeed - here 3 in the 5th column

[  8.   4.   3.   4.   3.  40.  -1.   0.   1.]
[  6.   6.   6.   9.   3.  30.   0.   3.   3.]
[  7.   8.   8.   8.   3.  35.   0.   0.   0.]
[  6.   1.   5.  10.   3.  30.   4.   9.   5.]
[  4.   5.   7.   7.   2.  20.   2.   2.   0.]


note that we impute the missign values to the test set using what we have learned frim the training set

In [38]:
Y = imp.transform(tt_c_no_label_num)
ppa(Y,3)

[  4.   3.   3.   7.   3.  20.   0.   4.   4.]
[  7.   8.   7.   7.   3.  35.  -1.  -1.   0.]
[  7.   4.   3.   7.   3.  35.  -1.   3.   4.]


we put back our transformed data in a dataframe

In [63]:
tr_c_no_label_num_tr = pd.DataFrame(X,columns=tr_c_no_label_num.columns,
                                           index = list(tr_c_no_label_num.index.values))
tr_c_no_label_num_tr.head()

,ex,s1,s2,s3,r23,exx,s21,s31,s32
12,8.0,4.0,3.0,4.0,3.0,40.0,-1.0,0.0,1.0
17,6.0,6.0,6.0,9.0,3.0,30.0,0.0,3.0,3.0
22,7.0,8.0,8.0,8.0,3.0,35.0,0.0,0.0,0.0
11,6.0,1.0,5.0,10.0,3.0,30.0,4.0,9.0,5.0
13,4.0,5.0,7.0,7.0,2.0,20.0,2.0,2.0,0.0


same thing for the test set

In [40]:
tt_c_no_label_num_tr = pd.DataFrame(Y,columns=tt_c_no_label_num.columns,
                                           index = list(tt_c_no_label_num.index.values))
tt_c_no_label_num_tr.head()

,ex,s1,s2,s3,r23,exx,s21,s31,s32
9,4.0,3.0,3.0,7.0,3.0,20.0,0.0,4.0,4.0
25,7.0,8.0,7.0,7.0,3.0,35.0,-1.0,-1.0,0.0
8,7.0,4.0,3.0,7.0,3.0,35.0,-1.0,3.0,4.0
21,8.0,7.0,5.0,7.0,3.0,40.0,-2.0,0.0,2.0
0,6.0,3.0,0.0,5.0,2.0,30.0,-3.0,2.0,5.0


<h3> Select and train a model </h3>

In [61]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression() # the standard linear regressor
lin_reg2 = LinearRegression(normalize=True) # a normalised version of linear regression

In [62]:
lin_reg.fit(tr_c_no_label_num_tr, tr_c_labels)
lin_reg2.fit(tr_c_no_label_num_tr, tr_c_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

<h4>ok we are good to go, the linear regressor has learned its best parameters, and we can predict now and compare with the actual values:</h4>

In [43]:
train_pred = lin_reg.predict(tr_c_no_label_num_tr)
print(train_pred)
deviation = train_pred - tr_c_labels # the total error vector, a numpy array

[ 7.62514481  9.00617147  8.76775554  8.8234535   9.19444173  8.0934536
  8.72545618  9.44956088  8.31433521  8.80555827  8.53921261  8.09212167
  8.44607282  8.59003503  8.01269633  7.82880436  8.21884382  8.36296396
  8.26984217  8.28471263  8.55275113  8.3206646   8.67594769]


In [44]:
# train_pred.shape[0] = 23 entrees 
train_pred.shape[0]

23

In [45]:
print("Deviations:\n")
print(deviation.sort_values(ascending=False))

Deviations:

23    2.092122
24    1.012696
11    0.823453
22    0.767756
1     0.725456
6     0.675948
3     0.446073
19    0.320665
27    0.218844
13    0.194442
17    0.006171
18   -0.171196
2    -0.194442
12   -0.374855
14   -0.447249
16   -0.460787
4    -0.550439
20   -0.637036
5    -0.685665
10   -0.715287
7    -0.730158
15   -0.906546
26   -1.409965
Name: d1, dtype: float64


<h4> now we compare the prediction and actual values by hand - using again rmse and lmse</h4>

follows bunch of scratch calculations to compute lmse and rmse by hand (to check that we are computing the right thing)

In [46]:
y = np.sqrt(1/train_pred.shape[0])
y # le coefficient de normalisation de la norme L2

0.20851441405707477

In [47]:
# sqrt(mean square error)
print("Rmse = ", y * np.linalg.norm(deviation))
# mean linear error
print("Lmse = ", sum(abs(deviation))/train_pred.shape[0])

Rmse =  0.771624629307
Lmse =  0.633358725171


In [48]:
# pour expliciter le calcul ci-dessus
print(sum(abs(deviation)))
res = 0
for x in list(deviation):
    res += abs(x)
print(res)

14.5672506789
14.5672506789


or we can compare the prediction and actual values with scikitlearn - using again rmse and lmse

In [49]:
# on retrouve bien la meme chose en invoquant la fonction predefinie de sklearn
from sklearn.metrics import mean_squared_error

In [50]:
lin_mse = mean_squared_error(train_pred,tr_c_labels)
lin_rmse = np.sqrt(lin_mse)
print("Rmse =",lin_rmse)
from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(train_pred,tr_c_labels)
print("Lmse =",lin_mae)

Rmse = 0.771624629307
Lmse = 0.633358725171


indeed we find the same!

Using the normalised version of the predictor makes no difference:

In [52]:
train_pred2 = lin_reg2.predict(tr_c_no_label_num_tr)
print(train_pred2 - train_pred)

[ -1.77635684e-15  -1.77635684e-15   0.00000000e+00  -1.77635684e-15
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -1.77635684e-15
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -1.77635684e-15
   0.00000000e+00  -1.77635684e-15  -1.77635684e-15  -8.88178420e-16
  -1.77635684e-15  -1.77635684e-15   0.00000000e+00  -1.77635684e-15
  -1.77635684e-15   0.00000000e+00   0.00000000e+00]


<h4>computing the scores of our two predictors</h4>

In [53]:
lin_reg.score(tr_c_no_label_num_tr,tr_c_labels)

0.2280171157738583

In [54]:
lin_reg.score(tt_c_no_label_num_tr,tt_c_labels)

0.011432878398945912

In [117]:
lin_reg2.score(tr_c_no_label_num_tr,tr_c_labels)

0.2280171157738583

In [56]:
lin_reg2.score(tt_c_no_label_num_tr,tt_c_labels)

0.011432878398946245

we see that indeed the scores are the same; also the score against the train set is better because it is easier to predict what one already knows :)

<h3> Using another predictor: decision trees </h3>

In [57]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(tr_c_no_label_num_tr,tr_c_labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [58]:
train_pred_tree = tree_reg.predict(tr_c_no_label_num_tr)
tree_mse = mean_squared_error(train_pred_tree,tr_c_labels)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

OK but ...

In [59]:
test_pred_tree = tree_reg.predict(tt_c_no_label_num_tr)
tree_mse = mean_squared_error(test_pred_tree,tt_c_labels)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

1.8973665961010275

this is called "overfitting" - the tree model is worse than the linear regressor on the test data